In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else 'cpu'

c:\Users\creep\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\cuda\__init__.py:82: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# Pobranie danych
trainigSet = torchvision.datasets.StanfordCars(
    root='data',
    split='train',
    download=False
)

tesingSet = torchvision.datasets.StanfordCars(
    root='data',
    split='test',
    download=False
)

In [4]:
classesDict = str(trainigSet.class_to_idx).split(',')
for i in classesDict:
    print(i)

{'AM General Hummer SUV 2000': 0
 'Acura RL Sedan 2012': 1
 'Acura TL Sedan 2012': 2
 'Acura TL Type-S 2008': 3
 'Acura TSX Sedan 2012': 4
 'Acura Integra Type R 2001': 5
 'Acura ZDX Hatchback 2012': 6
 'Aston Martin V8 Vantage Convertible 2012': 7
 'Aston Martin V8 Vantage Coupe 2012': 8
 'Aston Martin Virage Convertible 2012': 9
 'Aston Martin Virage Coupe 2012': 10
 'Audi RS 4 Convertible 2008': 11
 'Audi A5 Coupe 2012': 12
 'Audi TTS Coupe 2012': 13
 'Audi R8 Coupe 2012': 14
 'Audi V8 Sedan 1994': 15
 'Audi 100 Sedan 1994': 16
 'Audi 100 Wagon 1994': 17
 'Audi TT Hatchback 2011': 18
 'Audi S6 Sedan 2011': 19
 'Audi S5 Convertible 2012': 20
 'Audi S5 Coupe 2012': 21
 'Audi S4 Sedan 2012': 22
 'Audi S4 Sedan 2007': 23
 'Audi TT RS Coupe 2012': 24
 'BMW ActiveHybrid 5 Sedan 2012': 25
 'BMW 1 Series Convertible 2012': 26
 'BMW 1 Series Coupe 2012': 27
 'BMW 3 Series Sedan 2012': 28
 'BMW 3 Series Wagon 2012': 29
 'BMW 6 Series Convertible 2007': 30
 'BMW X5 SUV 2007': 31
 'BMW X6 SUV 2

In [5]:
import torchvision.transforms as T
rootDir = "ModelsAugV2Full/"
modelsTrainDir = rootDir + "Train/"

i = 1
for img, cls in trainigSet:

    augmentationFactor = 4
    words = classesDict[cls].split()
    category = words[len(words)-3]
    if category == 'Cab' or category == 'Convertible' or category == 'Coupe' or category == 'Hatchback' or category == 'Minivan' or category == 'Sedan' or category == 'SUV' or category == 'Van':
        if category == 'SuperCab':
            category = 'Cab'
            augmentationFactor = 2
        if category == 'Convertible':
            augmentationFactor = 2
        if category == 'Coupe':
            augmentationFactor = 2
        if category == 'Minivan':
            augmentationFactor = 8
        if category == 'Van':
            augmentationFactor = 8
        if category == 'Sedan':
            augmentationFactor = 1
        if category == 'SUV':
            augmentationFactor = 1
        if category == 'Hatchback':
            augmentationFactor = 4

        policies = [T.AutoAugmentPolicy.CIFAR10, T.AutoAugmentPolicy.IMAGENET, T.AutoAugmentPolicy.SVHN]
        augmenters = [T.AutoAugment(policy) for policy in policies]
        imgs = [
            [augmenter(img) for _ in range(augmentationFactor)]
            for augmenter in augmenters
        ]

        dir = 'data/' + modelsTrainDir + category + '/' + "%05d"%i + '.jpg'
        #print(dir)
        img.save(dir, 'JPEG')

        i = i+1

        for augImg in imgs:
            for j in range(augmentationFactor):
                dir = 'data/' + modelsTrainDir + category + '/' + "%05d"%i + '.jpg'
                augImg[j].save(dir, 'JPEG')
                i = i+1
    #break

Cab = 719       2
Conv = 1036     2
Coupe = 1054    2
Hatch = 554     4
Minivan = 250   8
Sedan = 1900    1
SUV = 1400      1
Van = 291       8

In [7]:
modelsTestDir = rootDir + "Test/"

classesDict = str(trainigSet.class_to_idx).split(',')
#for i in classesDict:
#    print(i, '\n')

i = 1
for img, cls in tesingSet:

    words = classesDict[cls].split()
    category = words[len(words)-3]
    if category == 'Cab' or category == 'Convertible' or category == 'Coupe' or category == 'Hatchback' or category == 'Minivan' or category == 'Sedan' or category == 'SUV' or category == 'Van':
        if category == 'SuperCab':
            category = 'Cab'
            augmentationFactor = 2
        if category == 'Convertible':
            augmentationFactor = 2
        if category == 'Coupe':
            augmentationFactor = 2
        if category == 'Minivan':
            augmentationFactor = 8
        if category == 'Van':
            augmentationFactor = 8
        if category == 'Sedan':
            augmentationFactor = 1
        if category == 'SUV':
            augmentationFactor = 1
        if category == 'Hatchback':
            augmentationFactor = 4
        dir = 'data/' + modelsTestDir + category + '/' + "%05d"%i + '.jpg'
        #print(dir)
        img.save(dir, 'JPEG')

        i = i+1
    #break